<a href="https://colab.research.google.com/github/rahatkader/Retinal-Vessels-Segmentation/blob/main/Retinal_Vessels_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Retinal Vessels Segmentation**

* Inspired from: https://github.com/CVxTz/medical_image_segmentation.git
* Dataset available at: https://www.isi.uu.nl/Research/Databases/DRIVE/

# Data Augmentation
Part of the script from aug_utils.py

### import

In [2]:
import numpy as np
import keras
from keras_preprocessing import image
from PIL import Image
import cv2

### random_flip

In [3]:
def random_flip(img, mask, u=0.5):
    if np.random.random() < u:    # np.random.random() Return random floats in the half-open interval [0.0, 1.0)
        img = image.flip_axis(img, 1)
        mask = image.flip_axis(mask, 1)
    if np.random.random() < u:
        img = image.flip_axis(img, 0)
        mask = image.flip_axis(mask, 0)
    return img, mask

### random_rotate

In [4]:
def random_rotate(img, mask, rotate_limit=(-20, 20), u=0.5):
    if np.random.random() < u:
        theta = np.random.uniform(rotate_limit[0], rotate_limit[1]) # np.random.uniform Draw samples from a uniform distribution
        img = image.apply_affine_transform(img, theta=theta)
        mask = image.apply_affine_transform(mask, theta=theta)
    return img, mask

### shift

In [5]:
def shift(x, wshift, hshift, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest', cval=0.):
    h, w = x.shape[row_axis], x.shape[col_axis]
    tx = hshift * h
    ty = wshift * w
    x = image.apply_affine_transform(x, ty=ty, tx=tx)
    return x

### random_shift

In [6]:
def random_shift(img, mask, w_limit=(-0.1, 0.1), h_limit=(-0.1, 0.1), u=0.5):
    if np.random.random() < u:
        wshift = np.random.uniform(w_limit[0], w_limit[1])
        hshift = np.random.uniform(h_limit[0], h_limit[1])
        img = shift(img, wshift, hshift)
        mask = shift(mask, wshift, hshift)
    return img, mask

### random_zoom

In [7]:
def random_zoom(img, mask, zoom_range=(0.8, 1), u=0.5):
    if np.random.random() < u:
        zx, zy = np.random.uniform(zoom_range[0], zoom_range[1], 2)
        img = image.apply_affine_transform(img, zx=zx, zy=zy)
        mask = image.apply_affine_transform(mask, zx=zx, zy=zy)
    return img, mask

### random_shear

In [8]:
def random_shear(img, mask, intensity_range=(-0.5, 0.5), u=0.5):
    if np.random.random() < u:
        sh = np.random.uniform(-intensity_range[0], intensity_range[1])
        img = image.apply_affine_transform(img, shear=sh)
        mask = image.apply_affine_transform(mask, shear=sh)
    return img, mask

### random_gray

In [9]:
def random_gray(img, u=0.5):
    if np.random.random() < u:
        coef = np.array([[[0.114, 0.587, 0.299]]])  # rgb to gray (YCbCr)
        gray = np.sum(img * coef, axis=2)
        img = np.dstack((gray, gray, gray)) # np.dstack Stack arrays in sequence depth wise
    return img

### random_contrast

In [10]:
def random_contrast(img, limit=(-0.3, 0.3), u=0.5):
    if np.random.random() < u:
        alpha = 1.0 + np.random.uniform(limit[0], limit[1])
        coef = np.array([[[0.114, 0.587, 0.299]]])  # rgb to gray (YCbCr)
        gray = img * coef
        gray = (3.0 * (1.0 - alpha) / gray.size) * np.sum(gray)
        img = alpha * img + gray
        img = np.clip(img, 0., 1.)
    return img

### random_brightness

In [11]:
def random_brightness(img, limit=(-0.3, 0.3), u=0.5):
    if np.random.random() < u:
        alpha = 1.0 + np.random.uniform(limit[0], limit[1])
        img = alpha * img
        img = np.clip(img, 0., 1.)
    return img

### random_saturation

In [12]:
def random_saturation(img, limit=(-0.3, 0.3), u=0.5): # randomly changing the intensity of the color
    if np.random.random() < u:
        alpha = 1.0 + np.random.uniform(limit[0], limit[1])
        coef = np.array([[[0.114, 0.587, 0.299]]])
        gray = img * coef
        gray = np.sum(gray, axis=2, keepdims=True)
        img = alpha * img + (1. - alpha) * gray
        img = np.clip(img, 0., 1.)
    return img

### random_channel_shift

In [13]:
def random_channel_shift(x, limit, channel_axis=2):
    x = np.rollaxis(x, channel_axis, 0)
    min_x, max_x = np.min(x), np.max(x)
    channel_images = [np.clip(x_ch + np.random.uniform(-limit, limit), min_x, max_x) for x_ch in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_axis + 1)
    return x

### random_crop

In [14]:
def random_crop(img, mask, u=0.1):
    if np.random.random() < u:
        w, h = img.shape[0], img.shape[1]
        offsetw = np.random.randint(w//2)
        offseth = np.random.randint(w//2)

        endw = np.random.randint(w // 2)+w // 2
        endh = np.random.randint(w // 2)+w // 2

        new_im = img[offsetw:offsetw + endw, offseth:offseth + endh, :]
        new_mask = mask[offsetw:offsetw + endw, offseth:offseth + endh, :]

        new_im, new_mask = cv2.resize(new_im, interpolation = cv2.INTER_LINEAR, dsize=(w, h)), \
               cv2.resize(new_mask, interpolation=cv2.INTER_LINEAR, dsize=(w, h))

        new_mask = new_mask[..., np.newaxis]
        return new_im, new_mask
    else:
        return img, mask

### random_augmentation

In [15]:
# random_brightness, random_contrast, random_saturation, random_rotate
# random_shear, random_flip, random_shift, random_zoom
def random_augmentation(img, mask):
    img = random_brightness(img, limit=(-0.1, 0.1), u=0.05)
    img = random_contrast(img, limit=(-0.1, 0.1), u=0.05)
    img = random_saturation(img, limit=(-0.1, 0.1), u=0.05)
    img, mask = random_rotate(img, mask, rotate_limit=(-10, 10), u=0.05)
    img, mask = random_shear(img, mask, intensity_range=(-5, 5), u=0.05)
    img, mask = random_flip(img, mask, u=0.5)
    img, mask = random_shift(img, mask, w_limit=(-0.1, 0.1), h_limit=(-0.1, 0.1), u=0.05)
    img, mask = random_zoom(img, mask, zoom_range=(0.9, 1.1), u=0.05)
    return img, mask

# Model Without Data Augmentation
This model is Trained from scratch without any data augmentation. Part of the script is from baseline.py. Architecture is taken from: https://github.com/jocicmarko/ultrasound-nerve-segmentation

### import

In [16]:
import argparse # command-line parsing module in the Python
from glob import glob # glob is used to return all file paths that match a specific pattern

import numpy as np
from PIL import Image
from keras import backend as K
from keras import losses
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Input, MaxPooling2D
from keras.layers import concatenate, Conv2D, Conv2DTranspose, Dropout, ReLU
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from numpy import random
import tensorflow as tf
#from aug_utils import random_augmentation
from random import randint

### Global variables

In [17]:
batch_size = 16
input_shape = (64, 64)

smooth = 1.

### custom_activation

In [18]:
def custom_activation(x):
    return K.relu(x, alpha=0.0, max_value=1)

### focal_loss

In [19]:
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

### get_unet

In [20]:
def get_unet(do=0, activation=ReLU):
    inputs = Input((None, None, 3)) # defining keras input layer
    conv1 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(inputs))) # padding same means zero padding
    conv1 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(conv1)))  # Dropout(0) layer randomly sets input units to 0
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(pool1)))
    conv2 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(conv2)))
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(pool2)))
    conv3 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(conv3)))
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(pool3)))
    conv4 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(conv4)))
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Dropout(do)(activation()(Conv2D(512, (3, 3), padding='same')(pool4)))
    conv5 = Dropout(do)(activation()(Conv2D(512, (3, 3), padding='same')(conv5)))

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3) 
    conv6 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(up6)))     # Conv2DTranspose= Transposed convolution layer (sometimes called Deconvolution).
    conv6 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(conv6)))   # concatenate= Layer that concatenates a list of inputs
                                                                                    # axis=3 means concatenates in 3rd dimension
    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(up7)))
    conv7 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(conv7)))

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(up8)))
    conv8 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(conv8)))

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(up9)))
    conv9 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(conv9)))

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-3), loss=losses.binary_crossentropy, metrics=['accuracy'])


    return model

### read_input

In [21]:
def read_input(path):
    x = np.array(Image.open(path))/255.   #binary image conversion
    return x

### read_gt

In [22]:
def read_gt(path):
    x = np.array(Image.open(path))/255. #binary image conversion
    return x[..., np.newaxis] # numpy.newaxis is used to increase the dimension
                              # Consecutive : can be replaced with ...
                              # here the input dimension is increased 

### random_crop

In [23]:
def random_crop(img, mask, crop_size=input_shape[0]):
    imgheight= img.shape[0]
    imgwidth = img.shape[1]
    i = randint(0, imgheight-crop_size)
    j = randint(0, imgwidth-crop_size)

    return img[i:(i+crop_size), j:(j+crop_size), :], mask[i:(i+crop_size), j:(j+crop_size)]

### gen

In [24]:
# read_input, read_gt, random_crop, random_augmentation
def gen(data, au=False):
    while True:
        repeat = 4
        index = random.choice(list(range(len(data))), batch_size // repeat)
        index = list(map(int, index))
        list_images_base = [read_input(data[i][0]) for i in index]
        list_gt_base = [read_gt(data[i][1]) for i in index]

        list_images = []
        list_gt = []

        for image, gt in zip(list_images_base, list_gt_base):

            for _ in range(repeat):
                image_, gt_ = random_crop(image.copy(), gt.copy())
                list_images.append(image_)
                list_gt.append(gt_)

        list_images_aug = []
        list_gt_aug = []

        for image, gt in zip(list_images, list_gt):
            if au:
                image, gt = random_augmentation(image, gt)
            list_images_aug.append(image)
            list_gt_aug.append(gt)

        yield np.array(list_images_aug), np.array(list_gt_aug)

### main

In [25]:
#  get_unet, 

if __name__ == '__main__':

    ap = argparse.ArgumentParser()
    ap.add_argument("-d", "--dropout", required=False, help="dropout", type=float, default=0.1)
    ap.add_argument("-a", "--activation", required=False, help="activation", default="ReLU")
    ap.add_argument("-f")

    args = vars(ap.parse_args())

    activation = globals()[args['activation']]

    model_name = "baseline_unet_do_%s_activation_%s_"%(args['dropout'], args['activation'])

    print("Model : %s\n\n"%model_name)

    train_data = list(zip(sorted(glob('/content/drive/MyDrive/Medical image segmentation/training/images/*.tif')),
                          sorted(glob('/content/drive/MyDrive/Medical image segmentation/training/1st_manual/*.gif'))))

    # calling get_unet
    model = get_unet(do=args['dropout'], activation=activation)

    file_path = model_name + "weights.best.hdf5"
    try:
        model.load_weights(file_path, by_name=True)
    except:
        pass




    history = model.fit_generator(gen(train_data, au=True), epochs=10, verbose=2,
                         steps_per_epoch= 10*len(train_data)//batch_size,
                                  use_multiprocessing=True, workers=16)

    #model.save_weights(file_path)

Model : baseline_unet_do_0.1_activation_ReLU_




/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
12/12 - 15s - loss: 0.6008 - accuracy: 0.7717 - 15s/epoch - 1s/step
Epoch 2/10
12/12 - 2s - loss: 0.4522 - accuracy: 0.8548 - 2s/epoch - 146ms/step
Epoch 3/10
12/12 - 6s - loss: 0.3326 - accuracy: 0.8933 - 6s/epoch - 498ms/step
Epoch 4/10
12/12 - 2s - loss: 0.3366 - accuracy: 0.8768 - 2s/epoch - 150ms/step
Epoch 5/10
12/12 - 7s - loss: 0.3060 - accuracy: 0.8966 - 7s/epoch - 618ms/step
Epoch 6/10
12/12 - 7s - loss: 0.3213 - accuracy: 0.8889 - 7s/epoch - 554ms/step
Epoch 7/10
12/12 - 4s - loss: 0.3150 - accuracy: 0.8903 - 4s/epoch - 309ms/step
Epoch 8/10
12/12 - 1s - loss: 0.3480 - accuracy: 0.8742 - 802ms/epoch - 67ms/step
Epoch 9/10
12/12 - 3s - loss: 0.2950 - accuracy: 0.9056 - 3s/epoch - 264ms/step
Epoch 10/10
12/12 - 3s - loss: 0.2901 - accuracy: 0.8978 - 3s/epoch - 229ms/step


In [26]:
model.save_weights(file_path)

# Predicting Model Without Data Augmentation

In this part model without data augmentation is being evaluted. Part of the script is from baseline_predict.py

### import

In [27]:
#from baseline import get_unet
from glob import glob
from PIL import Image
from skimage.transform import resize
import numpy as np
from matplotlib import pyplot as plt
from skimage.morphology import label
from pycocotools import mask as maskUtils
from tqdm import tqdm
import os
import cv2
from keras.layers import ReLU
from sklearn.metrics import roc_auc_score

### global variable

In [28]:
batchsize = 4
input_shape = (576, 576)

### batch

In [29]:
def batch(iterable, n=batchsize):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

### read_input

In [ ]:
#def read_input(path):
    #x = np.array(Image.open(path))/255.
    #return x

### read_gt

In [30]:
def read_gt(path):
    x = np.array(Image.open(path))
    return x[..., np.newaxis]/np.max(x)

### main

In [31]:
if __name__ == '__main__':
    model_name = "baseline_unet_do_0.1_activation_ReLU_"


    '''val_data = list(zip(sorted(glob('/content/drive/MyDrive/Medical image segmentation/test/images/*.tif')),
                          #sorted(glob('../input/DRIVE/test/2nd_manual/*.gif')),
                        sorted(glob('/content/drive/MyDrive/Medical image segmentation/test/mask/*.gif'))))'''

    val_data = list(zip(sorted(glob('/content/drive/MyDrive/Medical image segmentation/training/images/*.tif')),
                          sorted(glob('/content/drive/MyDrive/Medical image segmentation/training/1st_manual/*.gif')),
                        sorted(glob('/content/drive/MyDrive/Medical image segmentation/training/mask/*.gif'))))

    try:
        os.makedirs("/content/"+model_name+"test/", exist_ok=True)
    except:
        pass

    model = get_unet(do=0.1, activation=ReLU)

    file_path = "/content/baseline_unet_do_0.1_activation_ReLU_weights.best.hdf5" #file_path = model_name + "weights.best.hdf5"

    model.load_weights(file_path, by_name=True)

    gt_list = []
    pred_list = []

    for batch_files in tqdm(batch(val_data), total=len(val_data)//batchsize):

        imgs = [resize(read_input(image_path[0]), input_shape) for image_path in batch_files]
        seg = [read_gt(image_path[1]) for image_path in batch_files]
        mask = [read_gt(image_path[2]) for image_path in batch_files]

        imgs = np.array(imgs)

        pred = model.predict(imgs)

        pred_all = (pred)

        pred = np.clip(pred, 0, 1)

        for i, image_path in enumerate(batch_files):

            pred_ = pred[i, :, :, 0]

            pred_ = resize(pred_, (584, 565))

            mask_ = mask[i]

            gt_ = (seg[i]>0.5).astype(int)

            gt_flat = []
            pred_flat = []

            for p in range(pred_.shape[0]):
                for q in range(pred_.shape[1]):
                    if mask_[p,q]>0.5: # Inside the mask pixels only
                        gt_flat.append(gt_[p,q])
                        pred_flat.append(pred_[p,q])

            print(pred_.size, len(gt_list))

            gt_list += gt_flat
            pred_list += pred_flat

            pred_ = 255.*(pred_ - np.min(pred_))/(np.max(pred_)-np.min(pred_))

            image_base = image_path[0].split("/")[-1]

            cv2.imwrite("/content/"+model_name+"test/"+image_base, pred_)

    print(len(gt_list), len(pred_list))
    print("AUC ROC : ", roc_auc_score(gt_list, pred_list))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
  0%|          | 0/5 [00:00<?, ?it/s]

329960 0
329960 225600
329960 453286


 20%|██        | 1/5 [00:09<00:36,  9.25s/it]

329960 681473
329960 909199
329960 1136561
329960 1361653


 40%|████      | 2/5 [00:15<00:22,  7.54s/it]

...

 80%|████████  | 4/5 [00:29<00:07,  7.17s/it]

329960 3405945
329960 3633162
329960 3860348
329960 4086572


100%|██████████| 5/5 [00:36<00:00,  7.28s/it]

329960 4314032
4541006 4541006


AUC ROC :  0.4351053011069306


# Model With Pre-trained on ImageNet VGG Encoder + Data Augmentation
This model is Trained on pre-trained on ImageNet VGG encoder and data augmentation. Part of the script is from baseline_aug_vgg.py

Architecture is taken from: https://github.com/jocicmarko/ultrasound-nerve-segmentation


In [33]:
import argparse
from glob import glob

import numpy as np
from PIL import Image
from keras import backend as K
from keras import losses
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Input, MaxPooling2D
from keras.layers import concatenate, Conv2D, Conv2DTranspose, Dropout, ReLU
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from numpy import random
import tensorflow as tf
#from aug_utils import random_augmentation
from random import randint

In [34]:
batch_size = 16
input_shape = (64, 64)
smooth = 1.
VGG_PATH = "/content/drive/MyDrive/Medical image segmentation/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
#def custom_activation(x):
    #return K.relu(x, alpha=0.0, max_value=1)

In [ ]:
'''def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed'''

In [35]:
def get_unet(do=0, activation=ReLU):
    inputs = Input((None, None, 3))
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    conv1 = x
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    x = Dropout(do)(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    conv2 = x
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    x = Dropout(do)(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    conv3 = x
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    x = Dropout(do)(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    conv4 = x
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    x = Dropout(do)(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    conv5 = x
    x = Dropout(do)(x)

    vgg = Model(inputs, x)
    vgg.load_weights(VGG_PATH, by_name=True)
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(up6)))
    conv6 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(conv6)))

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(up7)))
    conv7 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(conv7)))

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(up8)))
    conv8 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(conv8)))

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(up9)))
    conv9 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(conv9)))

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-3), loss=losses.binary_crossentropy, metrics=['accuracy'])


    return model

In [ ]:
#def read_input(path):
    #x = np.array(Image.open(path))/255.
    #return x

In [36]:
#same as basline.py
def read_gt(path):
    x = np.array(Image.open(path))/255.
    return x[..., np.newaxis]

In [37]:
def random_crop(img, mask, crop_size=input_shape[0]):
    imgheight= img.shape[0]
    imgwidth = img.shape[1]
    i = randint(0, imgheight-crop_size)
    j = randint(0, imgwidth-crop_size)

    return img[i:(i+crop_size), j:(j+crop_size), :], mask[i:(i+crop_size), j:(j+crop_size)]

In [38]:
def gen(data, au=False):
    while True:
        repeat = 4
        index = random.choice(list(range(len(data))), batch_size // repeat)
        index = list(map(int, index))
        list_images_base = [read_input(data[i][0]) for i in index]
        list_gt_base = [read_gt(data[i][1]) for i in index]

        list_images_aug = []
        list_gt_aug = []

        for image, gt in zip(list_images_base, list_gt_base):
            if au:
                image, gt = random_augmentation(image, gt)
            list_images_aug.append(image)
            list_gt_aug.append(gt)

        list_images = []
        list_gt = []

        for image, gt in zip(list_images_aug, list_gt_aug):

            for _ in range(repeat):
                image_, gt_ = random_crop(image, gt)
                list_images.append(image_)
                list_gt.append(gt_)

        yield np.array(list_images), np.array(list_gt)

### main

In [39]:
if __name__ == '__main__':

    ap = argparse.ArgumentParser()
    ap.add_argument("-d", "--dropout", required=False, help="dropout", type=float, default=0.1)
    ap.add_argument("-a", "--activation", required=False, help="activation", default="ReLU")
    ap.add_argument("-f")

    args = vars(ap.parse_args())

    activation = globals()[args['activation']]

    model_name = "baseline_unet_aug_vgg_do_%s_activation_%s_"%(args['dropout'], args['activation'])

    print("Model : %s"%model_name)

    train_data = list(zip(sorted(glob('/content/drive/MyDrive/Medical image segmentation/training/images/*.tif')),
                          sorted(glob('/content/drive/MyDrive/Medical image segmentation/training/1st_manual/*.gif'))))


    model = get_unet(do=args['dropout'], activation=activation)

    file_path = model_name + "weights.best.hdf5"
    try:
        model.load_weights(file_path, by_name=True)
    except:
        pass




    history = model.fit_generator(gen(train_data, au=True), epochs=10, verbose=2,
                         steps_per_epoch= 10*len(train_data)//batch_size,
                                  use_multiprocessing=True, workers=16)

    #model.save_weights(file_path)

Model : baseline_unet_aug_vgg_do_0.1_activation_ReLU_
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


12/12 - 5s - loss: 0.4316 - accuracy: 0.8487 - 5s/epoch - 395ms/step
Epoch 2/10
12/12 - 3s - loss: 0.3045 - accuracy: 0.8944 - 3s/epoch - 249ms/step
Epoch 3/10
12/12 - 3s - loss: 0.2917 - accuracy: 0.8894 - 3s/epoch - 226ms/step
Epoch 4/10
12/12 - 2s - loss: 0.2490 - accuracy: 0.9047 - 2s/epoch - 173ms/step
Epoch 5/10
12/12 - 2s - loss: 0.2557 - accuracy: 0.9050 - 2s/epoch - 176ms/step
Epoch 6/10
12/12 - 3s - loss: 0.2540 - accuracy: 0.8986 - 3s/epoch - 235ms/step
Epoch 7/10
12/12 - 3s - loss: 0.1967 - accuracy: 0.9059 - 3s/epoch - 249ms/step
Epoch 8/10
12/12 - 1s - loss: 0.1840 - accuracy: 0.9169 - 1s/epoch - 119ms/step
Epoch 9/10
12/12 - 1s - loss: 0.1707 - accuracy: 0.9356 - 826ms/epoch - 69ms/step
Epoch 10/10
12/12 - 2s - loss: 0.1586 - accuracy: 0.9398 - 2s/epoch - 180ms/step


In [40]:
model.save_weights(file_path)

# Predicting Model With Pre-trained on ImageNet VGG Encoder + Data Augmentation.

In this part model with pre-trained on imageNet VGG encoder and data augmetation is being evaluted. Part of the script is from baseline_aug_vgg_predict.py.

In [41]:
#from baseline_aug_vgg import get_unet
from glob import glob
from cv2 import imread
from PIL import Image
from skimage.transform import resize
import numpy as np
from matplotlib import pyplot as plt
from skimage.morphology import label
from pycocotools import mask as maskUtils
from tqdm import tqdm
import os
import cv2
from keras.layers import ReLU
from sklearn.metrics import roc_auc_score

In [42]:
batchsize = 4
input_shape = (576, 576)

In [43]:
def batch(iterable, n=batchsize):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [44]:
def read_input(path):
    x = np.array(Image.open(path))/255.
    return x

In [45]:
def read_gt(path):
    x = np.array(Image.open(path))
    return x[..., np.newaxis]/np.max(x)

### main

In [46]:
if __name__ == '__main__':
    model_name = "baseline_unet_aug_vgg_do_0.1_activation_ReLU_"


    '''val_data = list(zip(sorted(glob('../input/DRIVE/test/images/*.tif')),
                          sorted(glob('../input/DRIVE/test/2nd_manual/*.gif')),
                        sorted(glob('../input/DRIVE/test/mask/*.gif'))))'''

    val_data = list(zip(sorted(glob('/content/drive/MyDrive/Medical image segmentation/training/images/*.tif')),
                          sorted(glob('/content/drive/MyDrive/Medical image segmentation/training/1st_manual/*.gif')),
                        sorted(glob('/content/drive/MyDrive/Medical image segmentation/training/mask/*.gif'))))

    try:
        os.makedirs("/content/"+model_name+"test/", exist_ok=True)
    except:
        pass

    model = get_unet(do=0.1, activation=ReLU)

    file_path = "/content/baseline_unet_aug_vgg_do_0.1_activation_ReLU_weights.best.hdf5" #model_name + "weights.best.hdf5"

    model.load_weights(file_path, by_name=True)

    gt_list = []
    pred_list = []

    for batch_files in tqdm(batch(val_data), total=len(val_data)//batchsize):

        imgs = [resize(read_input(image_path[0]), input_shape) for image_path in batch_files]
        seg = [read_gt(image_path[1]) for image_path in batch_files]
        mask = [read_gt(image_path[2]) for image_path in batch_files]

        imgs = np.array(imgs)

        pred = model.predict(imgs)

        pred_all = (pred)

        pred = np.clip(pred, 0, 1)

        for i, image_path in enumerate(batch_files):

            pred_ = pred[i, :, :, 0]

            pred_ = resize(pred_, (584, 565))

            mask_ = mask[i]

            gt_ = (seg[i]>0.5).astype(int)

            gt_flat = []
            pred_flat = []

            for p in range(pred_.shape[0]):
                for q in range(pred_.shape[1]):
                    if mask_[p,q]>0.5: # Inside the mask pixels only
                        gt_flat.append(gt_[p,q])
                        pred_flat.append(pred_[p,q])

            print(pred_.size, len(gt_list))

            gt_list += gt_flat
            pred_list += pred_flat

            pred_ = 255.*(pred_ - np.min(pred_))/(np.max(pred_)-np.min(pred_))

            image_base = image_path[0].split("/")[-1]

            cv2.imwrite("/content/"+model_name+"test/"+image_base, pred_)

    print(len(gt_list), len(pred_list))
    print("AUC ROC : ", roc_auc_score(gt_list, pred_list))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
  0%|          | 0/5 [00:00<?, ?it/s]

329960 0
329960 225600
329960 453286


 20%|██        | 1/5 [00:08<00:33,  8.33s/it]

329960 681473
329960 909199
329960 1136561
329960 1361653


 40%|████      | 2/5 [00:11<00:16,  5.37s/it]

...

 80%|████████  | 4/5 [00:18<00:04,  4.12s/it]

329960 3405945
329960 3633162
329960 3860348
329960 4086572


100%|██████████| 5/5 [00:21<00:00,  4.39s/it]

329960 4314032
4541006 4541006


AUC ROC :  0.5150285290619331
